# Kernel Support Vector Machines

We will implement a kernel SVM. Our implementation will be based on a generic quadratic programming optimizer provided in CVXOPT. The SVM will then be tested on the UCI breast cancer dataset, a simple binary classification dataset accessible via the `scikit-learn` library.

## Defining the Gaussian Kernel

As a starting point, we would like to implement the Gaussian kernel, which we will make use of in our kernel SVM implementation. It is defined as:
$$
k(x,x') = \exp \Big( -\frac{\|x-x'\|^2}{2 \sigma^2} \Big)
$$



In [9]:
#some imports
import numpy, scipy, scipy.spatial
import sklearn,sklearn.datasets
import cvxopt, cvxopt.solvers
cvxopt.solvers.options["show_progress"] = False

def GaussianKernel(X1, X2, s):
    return numpy.exp(
        -scipy.spatial.distance.cdist(X1, X2, "sqeuclidean") / (2 * (s ** 2))
    )


## Formulating the problem compatible to CVXOPT Quadratic Solver

We would like to learn a nonlinear SVM by optimizing its dual. An advantage of the dual SVM compared to the primal SVM is that it allows to use nonlinear kernels such as the Gaussian kernel. The dual SVM consists of solving the following quadratic program:

$$
\max_\alpha \sum_{i=1}^N \alpha_i - \frac12 \sum_{ij} \alpha_i \alpha_j y_i y_j k(x_i,x_j)
\qquad 
\text{subject to:}
\qquad 0 \leq \alpha_i \leq C \qquad \text{and} \qquad \sum_{i=1}^N \alpha_i y_i = 0.
$$

We would like to rely on a CVXOPT solver to obtain a solution to our SVM dual. QP solves an optimization problem of the type:
$$
\begin{align*}
\min_{\boldsymbol{x}} \quad &\frac12 \boldsymbol{x}^\top P \boldsymbol{x} + \boldsymbol{q}^\top \boldsymbol{x}\\
\text{subject to} \quad & G \boldsymbol{x} \preceq \boldsymbol{h}\\
\text{and} \quad & A \boldsymbol{x} = \boldsymbol{b}.
\end{align*}
$$
which is of similar form to our dual SVM (note that $\boldsymbol{x}$ will correspond to the parameters $(\alpha_i)_i$ of the SVM). We need to build the data structures (vectors and matrices) that makes solving this quadratic problem equivalent to solving our dual SVM:

$$
\begin{align*}
P=y^Ty K \in R^{NxN}\\
q=-\boldsymbol{1} \in R^N\\
G=\begin{pmatrix} 
-\boldsymbol{I}\\ 
\boldsymbol{I}
\end{pmatrix} \in R^{NxN}\\
h=C*\begin{pmatrix} 
\boldsymbol{0}\\ 
\boldsymbol{1}
\end{pmatrix} \in R^{2N}\\
A=y^T\\
b=0
\end{align*}
$$

with $\boldsymbol{1}$ and $\boldsymbol{0}$ a $R^N$ vector of all ones/zeros

In [2]:
def getQPMatrices(K, T, C):
    n_samples = len(K)
    P = cvxopt.matrix(numpy.outer(T, T) * K)
    q = cvxopt.matrix(numpy.ones(n_samples) * -1)
    G = cvxopt.matrix(numpy.vstack((
        -numpy.identity(n_samples), 
        numpy.identity(n_samples))
    ))
    h = cvxopt.matrix(numpy.hstack((
        numpy.zeros(n_samples), 
        numpy.ones(n_samples))
    )) * C
    A = cvxopt.matrix(T, (1, n_samples))
    b = cvxopt.matrix(0.0)

    return P, q, G, h, A, b



Given the parameters $(\alpha_i)_i$ the optimization procedure has found, the prediction of the SVM is given by:

$$
f(x) = \text{sign}\Big(\sum_{i=1}^N \alpha_i y_i k(x,x_i) + \theta\Big)
$$

With the bias $\theta$ obtained from any support vector that lies exactly on the margin, or equivalently, whose associated parameter $\alpha$ is not equal to $0$ or $C$. Calling one such vector "$x_M$", the parameter $\theta$ can be computed as:

$$
\theta =  y_M - \sum_{j=1}^N \alpha_j y_j k(x_M,x_j) 
$$

In [6]:
def Theta(K, T, alpha, C):
    # picking the vector
    m = numpy.argmax(
        numpy.logical_and(
            ~numpy.isclose(alpha, 0, atol=1e-5), 
            ~numpy.isclose(alpha, C, atol=1e-5)
        )
    )
    # too lazy to vectorize, no significant impact on runtime
    theta = T[m] - sum((alpha[j] * T[j] * K[m, j] for j in range(len(K))))
    return theta


## GaussianSVM

Putting all together, we can create a GaussianSVM class with a fit and a predict function:

In [7]:
class GaussianSVM:
    def __init__(self, C=1.0, scale=1.0):

        self.C, self.scale = C, scale

    def fit(self, X, T):
        K = GaussianKernel(X, X, self.scale)
        P, q, G, h, A, b = getQPMatrices(K, T, self.C)
        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        alpha = numpy.array(sol["x"])
        self.theta = Theta(K, T, alpha, self.C)
        #alpha!=0 is support vector, test with some margin to account for precision
        issupport = alpha[:, 0] > 1e-6 * alpha.mean()
        self.alpha = alpha[issupport]
        self.X = X[issupport]
        self.T = T[issupport]

    def predict(self, X):
        k = GaussianKernel(X, self.X, self.scale)
        f = numpy.sum(self.alpha.T * self.T[None, :] * k, 1) + self.theta
        return numpy.sign(f)


## Test Classifier

The following code tests the SVM on some breast cancer binary classification dataset for a range of scale and soft-margin parameters.

In [8]:
%%time

D = sklearn.datasets.load_breast_cancer()
X = D['data']
T = (D['target']==1)*2.0-1.0

for scale in [30,100,300,1000,3000]:
    for C in [10,100,1000,10000]:
        
        acctrain,acctest,nbsvs = [],[],[]
        
        svm = GaussianSVM(C=C,scale=scale)
        
        for i in range(3):

            # Split the data
            R = numpy.random.mtrand.RandomState(i).permutation(len(X))
            Xtrain,Xtest = X[R[:len(R)//2]]*1,X[R[len(R)//2:]]*1
            Ttrain,Ttest = T[R[:len(R)//2]]*1,T[R[len(R)//2:]]*1

            # Train and test the SVM
            svm.fit(Xtrain,Ttrain)
            acctrain += [(svm.predict(Xtrain)==Ttrain).mean()]
            acctest  += [(svm.predict(Xtest)==Ttest).mean()]
            nbsvs += [len(svm.X)*1.0]

        print('scale=%9.1f  C=%9.1f  nSV: %4d  train: %.3f  test: %.3f'%(
            scale,C,numpy.mean(nbsvs),numpy.mean(acctrain),numpy.mean(acctest)))
    print('')

scale=     30.0  C=     10.0  nSV:  188  train: 0.999  test: 0.926
scale=     30.0  C=    100.0  nSV:  170  train: 1.000  test: 0.923
scale=     30.0  C=   1000.0  nSV:  177  train: 1.000  test: 0.923
scale=     30.0  C=  10000.0  nSV:  183  train: 1.000  test: 0.923

scale=    100.0  C=     10.0  nSV:  109  train: 0.961  test: 0.943
scale=    100.0  C=    100.0  nSV:   92  train: 0.986  test: 0.944
scale=    100.0  C=   1000.0  nSV:   77  train: 0.998  test: 0.927
scale=    100.0  C=  10000.0  nSV:   62  train: 1.000  test: 0.925

scale=    300.0  C=     10.0  nSV:  105  train: 0.937  test: 0.931
scale=    300.0  C=    100.0  nSV:   58  train: 0.961  test: 0.945
scale=    300.0  C=   1000.0  nSV:   40  train: 0.978  test: 0.953
scale=    300.0  C=  10000.0  nSV:   33  train: 0.975  test: 0.935

scale=   1000.0  C=     10.0  nSV:   67  train: 0.919  test: 0.922
scale=   1000.0  C=    100.0  nSV:   61  train: 0.927  test: 0.937
scale=   1000.0  C=   1000.0  nSV:   53  train: 0.873  test